In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import math

data_dir = "../data"
print(os.listdir(data_dir))

['group_match_data_train.zip', '.DS_Store', 'test_V2.zip', 'train_V2.zip']


# load preprocessed data 

team & match data from saved file.

In [4]:
group_match_data = \
pd.read_csv(os.path.join(data_dir,'group_match_data_train.zip'))

# model training

In [5]:
def fill_missing(train_feature_no_obj):
    # fill missing value with most_frequent
    imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent', verbose=0, copy=False)
    imputer.fit(train_feature_no_obj)
    train_feature_no_obj = imputer.transform(train_feature_no_obj)
    return train_feature_no_obj

In [ ]:
for x in group_match_data.keys():
    print(pd.value_counts(group_match_data[x].isnull()))

In [7]:
# group_match_data['winPlacePerc'][group_match_data['winPlacePerc'].isnull()]
# group_match_data.iloc[1760675]
group_match_data = group_match_data.dropna()

## Linear Regression

In [9]:
df_train = group_match_data

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

corr= df_train.corr()
Y_Column = 'winPlacePerc'
corr_T = corr[[Y_Column]]
ColumnList_Target = corr_T.index.tolist()
df_trial = df_train[ColumnList_Target]
ColumnList_Target.remove(Y_Column)

X_Column = ColumnList_Target

X_Train = df_train[X_Column]
Y_Train = df_train[Y_Column]

X_train, X_test, Y_train, Y_test = train_test_split(X_Train, Y_Train, test_size= 0.3, random_state= 0)

model = LinearRegression()
model.fit(X_train, Y_train)
y_pred= model.predict(X_test)

M = mean_absolute_error(Y_test, y_pred)
print('Mean Absolute Error :', M)

Mean Absolute Error : 0.11223462817853729


## adaboost

In [8]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error, make_scorer

In [16]:
def split_data(data, X_Column, Y_Column):
    
    bin_size = 47964 // 3;
    split_position = bin_size * 2
    test = data.query('matchId >= %d' % split_position).head()
    train = data.query('matchId < %d' % split_position).head()
    
    X_train = train[X_Column]
    Y_train = train[Y_Column]

    X_test = test[X_Column]
    Y_test = test[Y_Column]
    
    return X_train, X_test, Y_train, Y_test

def train_and_cross_vali(model_name, clf, X, y, n_jobs=4):
    scores = cross_val_score(clf, X, y, cv=5, n_jobs=n_jobs, scoring=make_scorer(mean_absolute_error))
    score = scores.mean()
    print(f"{model_name} crossvali score: {score}")

In [14]:
group_match_data = group_match_data.dropna()
df_train = group_match_data.query('matchType == "solo-fpp"')
Y_Column = 'winPlacePerc'
X_Column = df_train.keys().tolist()
X_Column.remove(Y_Column)
X_Column.remove('matchType')
X_data = df_train[X_Column]
Y_data = df_train[Y_Column]
X_train, X_test, Y_train, Y_test = split_data(df_train, X_Column, Y_Column)

In [17]:
clf = AdaBoostRegressor(n_estimators=50)
train_and_cross_vali('AdaBoost', clf, X_data, Y_data, n_jobs=8)

AdaBoost crossvali score: 0.08591537158146079


## SVM

In [19]:
from sklearn import svm

In [20]:
clf = svm.SVR(gamma='auto')
train_and_cross_vali('AdaBoost', clf, X_data, Y_data, n_jobs=8)

KeyboardInterrupt: 